In [15]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart
from openpyxl.chart import Reference
import discord
from discord import SyncWebhook

In [ ]:
class SalesReport:
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        self.df = None
        self.wb = None

    def load_dataset(self):
        dfraw = pd.read_excel(self.input_file)
        self.df = dfraw.pivot_table(index='Gender',
                                    columns='Product line',
                                    values='Total',
                                    aggfunc='sum').round()

    def save_dataframe_to_excel(self):
        self.df.to_excel(self.output_file,
                         sheet_name='Report',
                         startrow=4)

    def generate_sales_report(self):
        self.load_dataset()
        self.save_dataframe_to_excel()

    def load_workbook(self):
        self.wb = load_workbook(self.output_file)
        self.wb.active = self.wb['Report']

    def add_chart(self):
        min_column = self.wb.active.min_column
        max_column = self.wb.active.max_column
        min_row = self.wb.active.min_row
        max_row = self.wb.active.max_row

        barchart = BarChart()

        data = Reference(self.wb.active, 
                            min_col=min_column+1,
                            max_col=max_column,
                            min_row=min_row,
                            max_row=max_row)

        categories = Reference(self.wb.active,
                                min_col=min_column,
                                max_col=min_column,
                                min_row=min_row+1,
                                max_row=max_row)

        barchart.add_data(data, titles_from_data=True)
        barchart.set_categories(categories)


        self.wb.active.add_chart(barchart, 'B12')
        barchart.title = 'Sales berdasarkan Produk'
        barchart.style = 2

    def calculate_total_sales(self):
        max_column = self.wb.active.max_column
        max_row = self.wb.active.max_row
        min_row = self.wb.active.min_row
        alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        alphabet_excel = alphabet[:max_column]

        for i in alphabet_excel:
            if i != 'A':
                self.wb.active[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
                self.wb.active[f'{i}{max_row+1}'].style = 'Currency'

        self.wb.active[f'{alphabet_excel[0]}{max_row+1}'] = 'Total'

    def format_report(self):
        self.wb.active['A1'] = 'Sales Report'
        self.wb.active['A2'] = '2019'
        self.wb.active['A1'].font = Font('Arial', bold=True, size=20)
        self.wb.active['A2'].font = Font('Arial', bold=True, size=10)

    def save_workbook(self):
        self.wb.save(self.output_file)
        
    def send_to_discord():
        webhook = SyncWebhook.from_url(webhook_url)

        with open(file=output_file, mode='rb') as file:
            excel_file = discord.File(file)

        webhook.send('This is an automated report', 
                    username='Sales Bot', 
                    file=excel_file)

    def generate_report(self):
        self.load_workbook()
        self.add_chart()
        self.calculate_total_sales()
        self.format_report()
        self.save_workbook()
        self.send_to_discord()


In [ ]:
# Define the input and output file paths
input_file = 'input_data/supermarket_sales.xlsx'
output_file = 'data_output/report_penjualan_2019.xlsx'
webhook_url = 'https://discord.com/api/webhooks/1110435167950082048/InS9He_OydBw66PUkXSc8Ts91i9lX6ChriIlaY8rx_DW4JyKzaPXvtWphRyBZn9Buz05'

# Create an instance of the SalesReport class
report = SalesReport(input_file, output_file)

# Generate the sales report
report.generate_report()